In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

city_df = pd.read_csv('cities.csv')
pro_df = pd.read_csv('providers.csv')
station_df = pd.read_csv('stations.csv')
ticket_df = pd.read_csv('ticket_data.csv')

From First attempt, information is accurate for all carpooling services. So let's drop all rows that involve carpool and work with busses and trains using departure and arrival stations as reference points

In [2]:
ticket_df['middle_stations']

0                        NaN
1                  {149,418}
2        {798,798,6794,6246}
3        {798,798,6794,6246}
4                {5983,5983}
                ...         
74163              {408,408}
74164            {1105,1105}
74165            {1105,1105}
74166              {863,863}
74167              {863,863}
Name: middle_stations, Length: 74168, dtype: object

In [3]:
ticket_df['middle_stations']

0                        NaN
1                  {149,418}
2        {798,798,6794,6246}
3        {798,798,6794,6246}
4                {5983,5983}
                ...         
74163              {408,408}
74164            {1105,1105}
74165            {1105,1105}
74166              {863,863}
74167              {863,863}
Name: middle_stations, Length: 74168, dtype: object

Need to map middle start station to middle station

Thought Process: dropna to get rid of carpooling services, create a new table with the origin station, middle station and destination station. Then figure out a way to split the middle stations into an array of size 2 or 4

In [4]:
new_tickets_df = ticket_df.dropna()

In [5]:
new_tickets_df.drop(['id','search_ts','o_city','d_city','other_companies'],  axis = 1, inplace = True)
new_tickets_df.head()

c:\users\admin8\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,middle_stations
1,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,"{149,418}"
2,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 21:24:00+00,7400,"{798,798,6794,6246}"
3,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 11:02:00+00,13500,"{798,798,6794,6246}"
4,8381,5905.0,6495.0,2017-10-13 21:46:00+00,2017-10-14 19:32:00+00,7710,"{5983,5983}"
49,13,767.0,396.0,2017-10-06 05:25:00+00,2017-10-06 16:40:00+00,1990,"{932,932}"


What i am missing is the distance bewteen all middle stations. Lets convert the middle stations to an array that we can iterate over

In [6]:

#r for raw strings, [] for a set of characters, \w for a set of strings that contain word character, + more than one case
new_tickets_df['Mid_array'] = 0
new_tickets_df.head()

<ipython-input-6-2a18584edb39>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_tickets_df['Mid_array'] = 0


,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,middle_stations,Mid_array
1,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,"{149,418}",0
2,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 21:24:00+00,7400,"{798,798,6794,6246}",0
3,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 11:02:00+00,13500,"{798,798,6794,6246}",0
4,8381,5905.0,6495.0,2017-10-13 21:46:00+00,2017-10-14 19:32:00+00,7710,"{5983,5983}",0
49,13,767.0,396.0,2017-10-06 05:25:00+00,2017-10-06 16:40:00+00,1990,"{932,932}",0


In [7]:
x = []
for i, row in new_tickets_df.iterrows():
    #print(i)
    x.append(re.findall(r"[\w]+", row['middle_stations']))

new_tickets_df['Mid_array'] = x
#x = re.findall(r"[\w]+",ticket_df['middle_stations'][1])

<ipython-input-7-c6c59dde9447>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_tickets_df['Mid_array'] = x


In [8]:
new_tickets_df.head()

,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,middle_stations,Mid_array
1,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,"{149,418}","[149, 418]"
2,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 21:24:00+00,7400,"{798,798,6794,6246}","[798, 798, 6794, 6246]"
3,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 11:02:00+00,13500,"{798,798,6794,6246}","[798, 798, 6794, 6246]"
4,8381,5905.0,6495.0,2017-10-13 21:46:00+00,2017-10-14 19:32:00+00,7710,"{5983,5983}","[5983, 5983]"
49,13,767.0,396.0,2017-10-06 05:25:00+00,2017-10-06 16:40:00+00,1990,"{932,932}","[932, 932]"


In [9]:
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   return np.round(res, 2)

In [10]:
station_df.set_index('id',inplace = True)

True Distance = distance from o_station to middle_stations to d_station
Iterate thru every row and calculate distance

In [11]:

dist = []
calc = 0
for i, row in new_tickets_df.iterrows():
    ref_lat = station_df.iloc[int(row['o_station'])]['latitude']
    ref_long = station_df.iloc[int(row['o_station'])]['longitude']
    for j in row['Mid_array']:
        calc += haversine_distance(station_df.iloc[int(j)]['latitude'],station_df.iloc[int(j)]['longitude'],ref_lat,ref_long)
        ref_lat = station_df.iloc[int(j)]['latitude']
        ref_long = station_df.iloc[int(j)]['longitude']
    calc +=  haversine_distance(station_df.iloc[int(row['d_station'])]['latitude'],station_df.iloc[int(row['d_station'])]['longitude'],ref_lat,ref_long)
    dist.append(calc)
    calc = 0
        
        
        

In [12]:
len(dist)#sanitycheck

32727

In [13]:
new_tickets_df['Distance'] = dist
new_tickets_df.head()

<ipython-input-13-ed96e96642a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_tickets_df['Distance'] = dist


,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,middle_stations,Mid_array,Distance
1,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,"{149,418}","[149, 418]",741.32
2,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 21:24:00+00,7400,"{798,798,6794,6246}","[798, 798, 6794, 6246]",1439.50
3,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 11:02:00+00,13500,"{798,798,6794,6246}","[798, 798, 6794, 6246]",1439.50
4,8381,5905.0,6495.0,2017-10-13 21:46:00+00,2017-10-14 19:32:00+00,7710,"{5983,5983}","[5983, 5983]",679.72
49,13,767.0,396.0,2017-10-06 05:25:00+00,2017-10-06 16:40:00+00,1990,"{932,932}","[932, 932]",1317.05


In [14]:
new_tickets_df.drop(['middle_stations','Mid_array'], axis = 1, inplace = True)
new_tickets_df['arrival_ts'] = pd.to_datetime(new_tickets_df['arrival_ts'])
new_tickets_df['departure_ts'] = pd.to_datetime(new_tickets_df['departure_ts'])

c:\users\admin8\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-14-81bb7e152936>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_tickets_df['arrival_ts'] = pd.to_datetime(new_tickets_df['arrival_ts'])
<ipython-input-14-81bb7e152936>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [15]:
new_tickets_df['Duration'] = (new_tickets_df['arrival_ts'] - new_tickets_df['departure_ts']).astype('timedelta64[m]')
new_tickets_df.head()

<ipython-input-15-8d02d268e74c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_tickets_df['Duration'] = (new_tickets_df['arrival_ts'] - new_tickets_df['departure_ts']).astype('timedelta64[m]')


,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,Distance,Duration
1,9,63.0,1044.0,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,1450,741.32,1070.0
2,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 21:24:00+00:00,7400,1439.50,1917.0
3,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 11:02:00+00:00,13500,1439.50,1295.0
4,8381,5905.0,6495.0,2017-10-13 21:46:00+00:00,2017-10-14 19:32:00+00:00,7710,679.72,1306.0
49,13,767.0,396.0,2017-10-06 05:25:00+00:00,2017-10-06 16:40:00+00:00,1990,1317.05,675.0


In [16]:
problem2 = new_tickets_df.drop(['arrival_ts','departure_ts'],axis = 1)

In [17]:
problem2.head()

,company,o_station,d_station,price_in_cents,Distance,Duration
1,9,63.0,1044.0,1450,741.32,1070.0
2,8377,5905.0,6495.0,7400,1439.50,1917.0
3,8377,5905.0,6495.0,13500,1439.50,1295.0
4,8381,5905.0,6495.0,7710,679.72,1306.0
49,13,767.0,396.0,1990,1317.05,675.0


In [18]:
temp_df = pd.read_csv('problem2_part1.csv')
temp_df.head()

,o_city,d_city,transport_type,Average Price,Average Duration,Distance,Category
0,5,23,train,20320.00,618.80,1013.91,3
1,6,227,train,11755.00,822.50,490.84,2
2,6,504,bus,3104.21,564.53,394.69,2
3,6,504,carpooling,4050.00,395.00,394.69,2
4,6,504,train,6585.71,373.71,394.69,2


In [19]:
problem2_car_df = temp_df[temp_df['transport_type']== 'carpooling']
problem2_car_df.head()

,o_city,d_city,transport_type,Average Price,Average Duration,Distance,Category
3,6,504,carpooling,4050.00,395.00,394.69,2
7,6,845,carpooling,848.99,78.19,91.46,1
8,6,10480,carpooling,3833.33,350.00,379.08,2
12,11,200,carpooling,3566.67,330.00,312.06,2
15,11,424,carpooling,6266.67,547.78,596.06,2


Now we need to organise the data from problem2 and then merge it with problem2_car_fg



In [20]:
problem2 = problem2.set_index('company').join(pro_df.set_index('id')).drop(['company_id','provider_id','name','fullname','has_wifi','has_plug',
                                                                      'has_adjustable_seats','has_bicycle'], axis = 1)
problem2.head()

,o_station,d_station,price_in_cents,Distance,Duration,transport_type
9,63.0,1044.0,1450,741.32,1070.0,bus
9,10144.0,396.0,1990,1437.95,395.0,bus
9,10144.0,396.0,1800,1437.95,1850.0,bus
9,3.0,396.0,1590,1186.19,250.0,bus
9,10144.0,396.0,1100,1953.14,1505.0,bus


In [21]:
problem2['transport_type'].unique() #sanity check, there should only be bus and trains left

array(['bus', 'train'], dtype=object)

In [22]:
duration_agg = problem2.assign(Duration = problem2['Duration'].abs()).groupby(['o_station','d_station','transport_type'])['Duration'].agg(
[('MinDuration','min'),('MeanDuration','mean'),('MaxDuration','max')])
duration_agg.reset_index(inplace = True)

In [23]:
duration_agg.head()

,o_station,d_station,transport_type,MinDuration,MeanDuration,MaxDuration
0,3.0,39.0,bus,375.0,427.5,575.0
1,3.0,61.0,bus,1155.0,1155.0,1155.0
2,3.0,123.0,bus,1350.0,1350.0,1350.0
3,3.0,132.0,bus,300.0,390.0,555.0
4,3.0,189.0,bus,875.0,875.0,875.0


In [24]:
price_agg = problem2.assign(price_in_cents = problem2['price_in_cents'].abs()).groupby(['o_station','d_station','transport_type'])['price_in_cents'].agg(
[('MinPrice','min'),('MeanPrice','mean'),('MaxPrice','max')])
price_agg.reset_index(inplace = True)
price_agg.head()

,o_station,d_station,transport_type,MinPrice,MeanPrice,MaxPrice
0,3.0,39.0,bus,1200,2238.333333,3480
1,3.0,61.0,bus,4780,4780.000000,4780
2,3.0,123.0,bus,3600,3600.000000,3600
3,3.0,132.0,bus,1890,2150.000000,2280
4,3.0,189.0,bus,3680,3680.000000,3680


In [25]:
dist_agg = problem2.assign(Distance = problem2['Distance'].abs()).groupby(['o_station','d_station','transport_type'])['Distance'].agg(
[('Distance','mean')])
dist_agg.reset_index(inplace = True)
dist_agg.head()

,o_station,d_station,transport_type,Distance
0,3.0,39.0,bus,918.71
1,3.0,61.0,bus,2032.19
2,3.0,123.0,bus,1186.36
3,3.0,132.0,bus,904.68
4,3.0,189.0,bus,2010.60


In [26]:
x = pd.merge(price_agg, duration_agg, on=('o_station','d_station','transport_type')) 
x['o_station'] = x['o_station'].astype(int)
x['d_station'] = x['d_station'].astype(int)
x.head()

,o_station,d_station,transport_type,MinPrice,MeanPrice,MaxPrice,MinDuration,MeanDuration,MaxDuration
0,3,39,bus,1200,2238.333333,3480,375.0,427.5,575.0
1,3,61,bus,4780,4780.000000,4780,1155.0,1155.0,1155.0
2,3,123,bus,3600,3600.000000,3600,1350.0,1350.0,1350.0
3,3,132,bus,1890,2150.000000,2280,300.0,390.0,555.0
4,3,189,bus,3680,3680.000000,3680,875.0,875.0,875.0


In [27]:
x1 = pd.merge(x, dist_agg, on=('o_station','d_station','transport_type')) 

In [28]:
x1.head()

,o_station,d_station,transport_type,MinPrice,MeanPrice,MaxPrice,MinDuration,MeanDuration,MaxDuration,Distance
0,3,39,bus,1200,2238.333333,3480,375.0,427.5,575.0,918.71
1,3,61,bus,4780,4780.000000,4780,1155.0,1155.0,1155.0,2032.19
2,3,123,bus,3600,3600.000000,3600,1350.0,1350.0,1350.0,1186.36
3,3,132,bus,1890,2150.000000,2280,300.0,390.0,555.0,904.68
4,3,189,bus,3680,3680.000000,3680,875.0,875.0,875.0,2010.60


In [29]:
problem2_x1 = x1.drop(['MinPrice','MaxPrice','MinDuration','MaxDuration'],axis = 1)

In [30]:
problem2_x1.rename(columns = {'o_station': 'Origin', 'd_station': 'Destination', 'MeanPrice':'Average Price', 'MeanDuration': 'Average Duration'}, inplace = True)
problem2_car_df.rename(columns = {'o_city': 'Origin', 'd_city': 'Destination'}, inplace = True)

c:\users\admin8\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [31]:
#problem2_car_df.reset_index(inplace = True)

In [32]:
problem2_x1.head()

,Origin,Destination,transport_type,Average Price,Average Duration,Distance
0,3,39,bus,2238.333333,427.5,918.71
1,3,61,bus,4780.000000,1155.0,2032.19
2,3,123,bus,3600.000000,1350.0,1186.36
3,3,132,bus,2150.000000,390.0,904.68
4,3,189,bus,3680.000000,875.0,2010.60


All we need to do now is to categorise the distance, merge the 2 data frames and group them according to distance category

In [33]:
category = []
for row in problem2_x1.itertuples(index = False):
    if row[5]<200:
        category.append(1)
    elif row[5]>200 and row[5]< 800:
        category.append(2)
    elif row[5]>800 and row[5]< 2000:
        category.append(3)
    else:
        category.append(4)
        

In [34]:
problem2_x1['Category'] = category
problem2_x1.head()

,Origin,Destination,transport_type,Average Price,Average Duration,Distance,Category
0,3,39,bus,2238.333333,427.5,918.71,3
1,3,61,bus,4780.000000,1155.0,2032.19,4
2,3,123,bus,3600.000000,1350.0,1186.36,3
3,3,132,bus,2150.000000,390.0,904.68,3
4,3,189,bus,3680.000000,875.0,2010.60,4


In [35]:
problem2_x1['Average Price'] = problem2_x1['Average Price'].astype(int)

In [75]:
frames = [problem2_x1, problem2_car_df]
Final = pd.concat(frames)

In [57]:
test = Final.groupby(['Category', 'transport_type'])

In [62]:
test.get_group((1,'carpooling'))

,Origin,Destination,transport_type,Average Price,Average Duration,Distance,Category
7,6,845,carpooling,848.99,78.19,91.46,1
26,11,580,carpooling,1387.50,130.00,143.08,1
34,11,843,carpooling,606.67,61.00,58.96,1
54,21,132,carpooling,974.00,92.40,105.00,1
56,21,453,carpooling,1277.88,100.96,99.35,1
...,...,...,...,...,...,...,...
2681,10480,539,carpooling,1100.00,170.00,121.39,1
2687,10780,714,carpooling,1191.67,113.33,124.32,1
2689,10998,9197,carpooling,1625.00,160.00,194.40,1
2691,11616,453,carpooling,1200.00,80.00,93.63,1


In [64]:
test.get_group((2,'carpooling'))

,Origin,Destination,transport_type,Average Price,Average Duration,Distance,Category
3,6,504,carpooling,4050.00,395.00,394.69,2
8,6,10480,carpooling,3833.33,350.00,379.08,2
12,11,200,carpooling,3566.67,330.00,312.06,2
15,11,424,carpooling,6266.67,547.78,596.06,2
20,11,485,carpooling,2323.21,219.17,253.79,2
...,...,...,...,...,...,...,...
2693,11835,173,carpooling,7108.33,621.67,698.02,2
2695,11835,246,carpooling,2366.67,190.00,227.35,2
2696,11835,485,carpooling,3505.00,309.00,400.33,2
2698,11938,126,carpooling,4227.94,375.15,498.45,2


In [65]:
test.get_group(((3,'carpooling')))

,Origin,Destination,transport_type,Average Price,Average Duration,Distance,Category
17,11,453,carpooling,7900.00,655.00,811.17,3
141,46,843,carpooling,7270.00,624.00,831.80,3
171,57,148,carpooling,9050.00,745.00,868.47,3
203,76,277,carpooling,9350.00,840.00,1092.84,3
461,173,453,carpooling,8500.00,686.67,842.67,3
563,200,1094,carpooling,10050.00,1170.00,1458.86,3
637,317,23,carpooling,7100.00,610.00,803.31,3
682,367,467,carpooling,8600.00,690.00,1044.69,3
741,440,825,carpooling,6600.00,580.00,827.55,3
765,451,1918,carpooling,11250.00,1147.50,1754.30,3


In [67]:
test.get_group((1,'bus'))

,Origin,Destination,transport_type,Average Price,Average Duration,Distance,Category
1951,2376,1492,bus,1851.0,163.857143,97.55,1
1953,2384,1492,bus,1851.0,154.714286,102.20,1
1954,2388,1492,bus,1861.0,159.600000,105.36,1
1955,2390,1492,bus,1875.0,146.076923,97.43,1
1956,2391,1492,bus,1851.0,145.571429,96.85,1
1957,2392,1492,bus,1851.0,143.700000,98.35,1
1958,2393,1492,bus,1852.0,186.000000,96.92,1
2033,4151,6799,bus,3420.0,381.000000,181.00,1


In [68]:
test.get_group((2,'bus'))

,Origin,Destination,transport_type,Average Price,Average Duration,Distance,Category
6,3,296,bus,5850.0,560.000000,693.89,2
8,3,322,bus,3056.0,708.333333,473.58,2
15,3,489,bus,2000.0,595.000000,614.45,2
19,3,575,bus,1390.0,450.000000,798.66,2
59,12,870,bus,3550.0,590.000000,663.50,2
...,...,...,...,...,...,...,...
2728,10642,734,bus,2290.0,455.000000,783.38,2
2749,10642,3594,bus,3800.0,2005.000000,611.19,2
2752,10642,7903,bus,2300.0,925.000000,633.34,2
2755,10642,8858,bus,4400.0,795.000000,575.80,2


In [69]:
test.get_group((3,'bus'))

,Origin,Destination,transport_type,Average Price,Average Duration,Distance,Category
0,3,39,bus,2238.0,427.500000,918.710000,3
2,3,123,bus,3600.0,1350.000000,1186.360000,3
3,3,132,bus,2150.0,390.000000,904.680000,3
5,3,273,bus,4263.0,802.978723,1783.025532,3
7,3,304,bus,2344.0,1349.090909,1929.540000,3
...,...,...,...,...,...,...,...
2856,11012,11008,bus,6800.0,275.000000,1376.070000,3
2857,11012,11010,bus,9600.0,1605.000000,1614.670000,3
2858,11013,11005,bus,5000.0,210.000000,1342.030000,3
2859,11014,11005,bus,5000.0,210.000000,1394.210000,3


In [70]:
test.get_group((4,'bus'))

,Origin,Destination,transport_type,Average Price,Average Duration,Distance,Category
1,3,61,bus,4780.0,1155.000000,2032.190000,4
4,3,189,bus,3680.0,875.000000,2010.600000,4
21,3,581,bus,2430.0,610.000000,2303.620000,4
24,3,1047,bus,5690.0,450.000000,2390.910000,4
28,6,409,bus,3400.0,329.000000,2713.440000,4
...,...,...,...,...,...,...,...
2841,10887,897,bus,2100.0,1225.000000,2226.455000,4
2846,10894,586,bus,2490.0,746.666667,2211.590000,4
2847,10894,763,bus,7042.0,1438.000000,2188.768000,4
2848,10894,932,bus,1456.0,423.333333,2162.776667,4


In [71]:
test.get_group((1,'train'))

,Origin,Destination,transport_type,Average Price,Average Duration,Distance,Category
759,408,3772,train,11720.0,724.000000,159.26,1
2007,3772,408,train,11520.0,1447.000000,159.26,1
2034,4151,6799,train,5410.0,135.714286,177.97,1


In [72]:
test.get_group((2,'train'))

,Origin,Destination,transport_type,Average Price,Average Duration,Distance,Category
92,60,6833,train,2950.0,155.800000,542.580000,2
116,62,581,train,12542.0,1059.142857,754.010000,2
245,248,153,train,11250.0,527.800000,661.220000,2
262,248,572,train,6010.0,817.000000,680.920000,2
264,248,585,train,13077.0,455.629630,765.834074,2
...,...,...,...,...,...,...,...
2485,7957,6794,train,4258.0,595.800000,297.634000,2
2494,8033,396,train,9822.0,289.500000,700.409167,2
2499,8033,6797,train,10358.0,184.941176,524.584118,2
2548,9062,7524,train,1831.0,300.947368,684.532632,2


In [73]:
test.get_group((3,'train'))

,Origin,Destination,transport_type,Average Price,Average Duration,Distance,Category
53,12,568,train,16832.0,476.529412,1392.774706,3
65,12,6057,train,24360.0,278.800000,1492.230000,3
83,60,510,train,2210.0,162.444444,1921.791111,3
85,60,538,train,13606.0,356.666667,1634.547500,3
90,60,6794,train,5005.0,306.666667,1049.626667,3
...,...,...,...,...,...,...,...
2837,10876,7642,train,7555.0,231.888889,1545.466667,3
2843,10894,334,train,14595.0,637.500000,1237.693750,3
2849,10894,7205,train,4577.0,274.250000,1601.855000,3
2850,10895,279,train,5050.0,643.000000,1173.080000,3


In [74]:
test.get_group((4,'train'))

,Origin,Destination,transport_type,Average Price,Average Duration,Distance,Category
43,12,334,train,5162.0,151.500000,2644.175000,4
89,60,6246,train,7435.0,253.500000,2285.200000,4
91,60,6799,train,7406.0,173.333333,2223.135000,4
190,151,3893,train,12325.0,812.428571,3184.398571,4
191,151,4667,train,4116.0,309.685714,2577.126857,4
...,...,...,...,...,...,...,...
2824,10876,487,train,8778.0,825.888889,10720.673333,4
2832,10876,6560,train,13460.0,1073.800000,2049.070000,4
2834,10876,6567,train,10840.0,206.000000,18454.900000,4
2835,10876,6590,train,6412.0,218.000000,2287.782857,4
